In [5]:
import requests
import numpy as np
import csv
from bs4 import BeautifulSoup

In [6]:
class ScraperApi:
    
    def __init__(self,url):
        _key = "f510817b0c4b96aef3ebb365664c8dfc"
        self.key = _key
        self.url_scraper ="http://api.scraperapi.com?api_key=" + self.key + "&url=" + url
        print(self.url_scraper)
        
    def getSoup(self):
        response = requests.get(self.url_scraper)
        
        if response.status_code != 200:
            print('Error with your request: ' + str(response.status_code))
        
        else:
            print('there is a response')
            soup = BeautifulSoup(response.content, 'html.parser')
            return soup

In [7]:
class ScrapingBee:
    def __init__(self, url_site):
        
        # mamorina compte scrapingBee @gmail dia alaina iny key iny dia apetaka eto dia mandeha
        
        key = "24TOVPZDNT4YZC4NCYV73M27I1M00KHZOFFRDU223L1LA574CWAMJOUJRQAR5FTUW62AT5I6OCGGJB45"
        self.SCRAPINGBEE_API_KEY = key
        self.endpoint = "https://app.scrapingbee.com/api/v1"
        self.params = {
            'api_key': self.SCRAPINGBEE_API_KEY,
            'url': url_site,
            'render_js': 'false', 
        }
        
        self.url_site = url_site
        
        self.page = requests.get(self.url_site)
        
    def getSoup(self):
        response = requests.get(self.endpoint, params=self.params)
        
        if response.status_code != 200:
            print('Error with your request: ' + str(response.status_code))
        
        else:
            print('there is a response')
            soup = BeautifulSoup(response.content, 'html.parser')
            return soup
        
    def getSoupTraditionnal(self):
        soup = BeautifulSoup(self.page.content,'html.parser')
        return soup
        
        
class FileCsv:
    
    def __init__(self, _path):
        self.path = _path
        
    def setHeader(self,header):
        with open(self.path, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header)
        
    def addColumn(self, row):
        with open(self.path, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(row)
            
    def read(self, _path):
        with open(_path,'r') as f:
            spamreader = csv.reader(f, delimiter=',')
            for row in spamreader:
                print(row)
        

In [8]:

# scraping a page with all details
    
class ScrapingHouse:
    
    def __init__(self,url_house):
        self.url = url_house
        
        # sb = ScraperApi(url_house)
        sb = ScrapingBee(url_house)
        self.soup = sb.getSoup()

        self.features = {
            'page' : 0,
            'features' : [],
            'values' : []
        }
        self.path = ""
        
        self.liensVille = []
        
    def getData(self):
        
        if self.soup:
            
            features = self.soup.find_all('div',class_='p24_propertyOverviewKey')
            datas = self.soup.find_all('div',class_='p24_info')
            div_price = self.soup.find_all('div',class_="p24_mBM")
            
            if features and datas and div_price:
                
                price = div_price[0].find_all('div','p24_price')[0].text 
                
                for i in range(np.shape(datas)[0]):
                    self.features['values'].append(datas[i].text)
                    features = datas[i].parent.parent.div.text
                    self.features['features'].append(features)

                self.features['features'].append('price')
                self.features['values'].append(price)


                # adding the adress if exist

                a = self.soup.find('a',class_="p24_addressPropOverview")

                if a:
                    adress = a.text
                    features_adress = a.parent.parent.div.text
                    self.features['features'].append(features_adress)
                    self.features['values'].append(adress)

        print(self.features)
        
        
            
#         for i in range(np.shape(features)[0]):
# #             self.features['features'].append(features[i].text)
#             print(features[i].text)
    
#         print('--------------')
        
#         for i in range(np.shape(datas)[0]):
#             print(datas[i].text)
            
#         print(self.features)
        
#         for i in range(np.shape(datas)[0]):
#             self.features['values'].append(datas[i].text)
            
        return self.features
    
    def saveData(self,page):
        
        file = FileCsv('../data/house_price/' + page + '.csv')
        file.setHeader(self.features['features'])
        file.addColumn(self.features['values'])
    


In [9]:
class ScrapingProvince:
    
    def __init__(self,url_prov):
        self.url_page = []
        self.nb_page = 0
        self.page = requests.get(url_prov)
        self.url_prov = url_prov
    
    def findUrlPage(self):
        # sb = ScraperApi(self.url_prov)
        sb = ScrapingBee(self.url_prov)
        soup = sb.getSoup()
#         print(soup)
        
        ul_pagination = soup.find_all("ul",class_="pagination")
        li = ul_pagination[0].find_all("li")

        last_page = int(li[np.shape(li)[0]-1].a.get('data-pagenumber'))
        
        for i in range(last_page):
            split = self.url_prov.split('?')

            split[0] += '/p' + str(i+1) + '?'
            url = split[0] + split[1]
            self.url_page.append(url)
            
        print(self.url_page)
            
        return self.url_page
        
    def saveUrl(self,province):
        file = FileCsv('../data/province/page/' + province + '.csv')
        file.setHeader(self.features['features'])
        file.addColumn(self.features['values'])
        

In [10]:
class ScrapingPage:
    
    def __init__(self,url_page):
        self.url_page = url_page
        # self.soup = ScraperApi(url_page).getSoup()
        self.soup = ScrapingBee(url_page).getSoup()
        self.url_house = []
        
    def getAllUrlHouse(self):
        soup = self.soup
        div_container = soup.find_all('div','p24_regularTile')
        list_href = []
        
        for i in range(np.shape(div_container)[0]):
             list_href.append('https://www.property24.com' + div_container[i].a.get('href'))
        
        self.url_house = list_href
        
        print(self.url_house)
        
        return self.url_house
        

In [11]:
class Site:
    def __init__(self, url):
        self.url = url
        self.url_ville = []
        self.url_page_ville = []  # ato misy lien ana page bdb
        self.list_url_house = []
        self.url_page = []
        
    def findUrlVille(self):
        
        # soup = ScraperApi(self.url).getSoup() 
        soup = ScrapingBee(self.url).getSoupTraditionnal()
        
        div = soup.find_all('div',class_='p24_popular')
        
        file = FileCsv('../data/ville/url_ville.csv')
        file.setHeader(['url_ville'])
        
        for i in range(np.shape(div)[0]):
            a = div[i].find_all('a')
            for j in range(np.shape(a)[0]):
                if j== np.shape(a)[0]-3:
                    break
                else:
                    url = 'https://www.property24.com'+ a[j].get('href') + '?PropertyCategory=House%2cApartmentOrFlat'
                    self.url_ville.append(url)
                    file.addColumn([url])
        
    # prend en parametre un tableau d'url_ville et le parcours pour obtenir les liens_de son page
    # liens des (1......5689)
    
    def findUrlPageVille(self, url_ville):
        
        url_page = []
        
        for i in range(np.shape(url_ville)[0]):
            
            sp = ScrapingProvince(url_ville[i])
            url_page_one = sp.findUrlPage()
            url_page.append(url_page_one)
            self.url_page = url_page[0]
        
    def getContentPageVille(self,name_file):  # maka ny lien an'ilay house ao anaty ilay page
        
        file = FileCsv('../data/house_price/house_' + name_file + '.csv')
        file.setHeader(['data_house'])
        
        for i in range(np.shape(self.url_page)[0]):
            sp = ScrapingPage(self.url_page[i])
            
            allUrlHouse = sp.getAllUrlHouse()
            
            for j in range(np.shape(allUrlHouse)[0]):
                self.getDataHouse(allUrlHouse[j],file)
                print('ok j')
    
    def getDataHouse(self,url_house,file):
        
        print(url_house)
        
        sh = ScrapingHouse(url_house)
        features = sh.getData()
        
        file.addColumn(features['features'])
        file.addColumn(features['values'])
        
        
            
        

In [13]:
site= Site('https://www.property24.com')
print ('ok')
# --- atao valeur an'io tableau io ilay lien ana province tiana ho alaina dia mahazo ny house rehetra ao aminy--
# --- 

lien_province_tiana_alaina = 'https://www.property24.com/for-sale/randburg/gauteng/8?PropertyCategory=House%2cApartmentOrFlat'
url_ville = [lien_province_tiana_alaina]

site.findUrlPageVille(url_ville)

# ovana ilay name_file dia ville no aloha dia province no farany, raha /mabopane/gauteng izany ilay lien ohatra dia lasa
# mabopane_gauteng ilay fichier istoquer-na anazy

name_file = "randburg_gauteng"
site.getContentPageVille(name_file)


ok
there is a response
['https://www.property24.com/for-sale/randburg/gauteng/8/p1?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p2?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p3?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p4?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p5?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p6?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p7?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p8?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p9?PropertyCategory=House%2cApartmentOrFlat', 'https://www.property24.com/for-sale/randburg/gauteng/8/p1

there is a response
['https://www.property24.com/for-sale/jukskei-park/randburg/gauteng/4279/110287572', 'https://www.property24.com/for-sale/randpark-ridge/randburg/gauteng/4471/110287567', 'https://www.property24.com/for-sale/president-ridge/randburg/gauteng/4399/110287497', 'https://www.property24.com/for-sale/north-riding/randburg/gauteng/4455/110287477', 'https://www.property24.com/for-sale/northcliff/randburg/gauteng/5783/110287443', 'https://www.property24.com/for-sale/ferndale/randburg/gauteng/4406/110287391', 'https://www.property24.com/for-sale/victory-park/randburg/gauteng/5793/110287219', 'https://www.property24.com/for-sale/linden/randburg/gauteng/5779/110216098', 'https://www.property24.com/for-sale/nietgedacht-ah/randburg/gauteng/10358/110287057', 'https://www.property24.com/for-sale/ferndale/randburg/gauteng/4406/108902079', 'https://www.property24.com/for-sale/blackheath/randburg/gauteng/4541/110286760', 'https://www.property24.com/for-sale/northcliff/randburg/gauteng/

there is a response
{'page': 0, 'features': ['Listing Number', 'Type of Property', 'Description', 'Listing Date', 'Erf Size', 'Rates and Taxes', 'Pets Allowed', 'Bedrooms', 'Bedroom 1', 'Bedroom 2', 'Bedroom 3', 'Bedroom 4', 'Bedroom 5', 'Bedroom 6', 'Bedroom 7', 'Bedroom 8', 'Bathrooms', 'Kitchen', 'Office/study', 'Domestic Rooms', 'Lounges', 'Lounge 1', 'Lounge 2', 'Dining Rooms', 'Dining Room 1', 'Dining Room 2', 'Braai Room', 'Entrance Hall', 'Rooms', 'Garden', 'Pool', 'Security', 'Special Feature', 'Flatlet', 'price', 'Street Address'], 'values': ['110286653', 'House', 'Multi Storey', '10 August 2021', '4 433 m²', 'R 2 833', 'Yes', '8', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '7.5', 'Pantry, Scullery, Laundry', 'Study', '1', '2', ' ', ' ', '2', ' ', ' ', 'Braai Room', 'Entrance Hall', 'Patio', 'Yes', 'Yes', 'Alarm System, Intercom, Electric fencing', 'Built in Braai, Borehole', 'Yes', 'R 9 500 000', ' 13 Ebony Drive, Northcliff']}
ok j
https://www.property24.com/for-sale/bloubosra

KeyboardInterrupt: 

In [ ]:

class File:
    
    def __init__(self, path):
        self.path = path
        self.compt = 0
        
    def computeData(self):
        compt = 0
        with open(self.path,'r') as f:
            spamreader = csv.reader(f, delimiter=',')
            for row in spamreader:
                if row != []:
                    compt += 1
                                
        return compt//2
    
    def getFeatures(self):
        features = []
        self.compt = 0
        
        with open(self.path,'r') as f:
            spamreader = csv.reader(f, delimiter=',')
            
            for row in spamreader:
                
                if row == []:
                    a=1
                else:
                    self.compt += 1
                    
                    if self.compt == 2:
                        features += row
                    elif self.compt ==3 :
                        self.compt = 1    
                        
        return list(OrderedDict.fromkeys(features))
    

    def getOneRow(self,features,values,features_final):

        row = [[''] * np.shape(features_final)[0]] * 1
        
        n = np.shape(features)[0]
        
        # size features censé hitovy amin'ny size values
        
        for i in range(n):
            index_features = features_final.index(features[i])
            row[0][index_features] = values[i]
            
        return row
            
        
    # method miretourne ny ligne @csv final, nan izay tsy misy anazy , omena features ilay itambarany dia iny no iaingana
    
    
    def getRow(self,features_final):
        
        row_final = []
        
        with open(self.path,'r') as f:
            spamreader = csv.reader(f, delimiter=',')
            
            features = []
            values = []
            
            for row in spamreader:
                
                if row != []:
                    self.compt += 1
                    
                    if self.compt == 2:
                        features = row
                        
                    elif self.compt == 3 :
                        self.compt = 1
                        values = row
                        
                        row_final += self.getOneRow(features, values, features_final)
                        
        return row_final
                        
    
class Data:
    
    def __init__(self, list_path):
        self.list_path = list_path
        
    def computeData(self):
        nb = 0
        
        for i in range(np.shape(self.list_path)[0]):
            path = '../data/house/' + self.list_path[i]
            f = File(path)
            nb += f.computeData()
        
        return nb
    
    def getFeatures(self):
        
        features = []
        
        for i in range(np.shape(self.list_path)[0]):
            path = '../data/house/' + self.list_path[i]
            f = File(path)
            features += f.getFeatures()
            features = list(OrderedDict.fromkeys(features))
            
        return features
    
    def save_one_csv(self):
        
        features_final = self.getFeatures()
        row = []
        file = FileCsv("../data/all_data.csv");
        file.setHeader(features_final)
        
        for i in range(np.shape(self.list_path)[0]):
            path = '../data/house/' + self.list_path[i]
            f = File(path)
            row += f.getRow(features_final)
            
        for i in range(np.shape(row)[0]):
            file.addColumn(row[i])
            
        print("All Data saved in the file : data/all_data.csv")

Assemblage des fichiers en une seule et unique fichier 'all_data.csv'

In [ ]:
list_file = listdir("../data/house")

data = Data(list_file)
data.save_one_csv()